<center><h1><b>Subway Surfers </h1></center>



<p>
  <img src="https://i.ytimg.com/vi/qbo8aX-oIew/maxresdefault.jpg" alt="Logo" width="700"/>
</p>


# Project Overview
Experience "Subway Surfers," a data-driven adventure where your goal is to investigate the complex network that is the New York City subway system. Our crew navigates the digital rails like subway surfers, utilizing the vast network that is relied upon by 32% of New York City residents for their everyday commute.

Equipped with our digital surfboards, we set out on this adventure with the Hourly subway dataset—a gold mine that reveals the fluctuations in commuter traffic at particular times. It feels like you are experiencing the minute-by-minute rhythmic pulse of the city's heartbeat. Every data point is a station stop, a split second captured in time, eagerly awaiting our team's deciphering of the patterns that make this subway symphony tick.

But we're not simply riding the data waves. The Subway station information provides us with the precise coordinates of all 496 stations, transforming our surfboards into precision instruments for traversing the immense expanse of the metropolis. We're not simply analyzing; we're locating, examining, and revealing the subway network's geographic tapestry.


### Research Questions:

- Are there significant differences in frequency of use and passenger flow across different subway lines?

- How does subway ridership in New York City vary between weekdays and weekends across different time periods of the day?

- Do you predict specific holidays to cause more surges over others? For example, New Year's Eve and Thanksgiving.

- Which subway route experiences the highest level of crowding with respect to time?


### Datasets
The datasets are hosted on data.ny.gov <br>
#### 🚇 NYC Subway Station Ridership Dataset Overview <br>
This dataset provides subway ridership estimates on an hourly basis. It is composed of the following gleaming threads of data:
- 📅 transit_timestamp: Records the date and time when ridership transactions occur, rounded down to the nearest hour for uniformity.
- 🔑 station_complex_id: A unique alphanumeric code assigned to each subway station complex for identification.
- 🏷️ station_complex: The name of the subway station complex, noting that larger complexes may represent multiple lines.
- 🗺️ borough: Indicates which of the five boroughs the subway station serves, providing geographical segmentation.
- 🚇 routes: Details the subway lines that service each station, critical for route-specific analysis.
- 💲 payment_method: Specifies the fare medium, OMNY or MetroCard, used for entry into the system.
- 👤 ridership: Quantifies the total entries at each station, reflecting the volume of station use.
- 🔄 transfers: Counts the number of free transfers at each station, a subset of the total ridership.
- 🌐 latitude, longitude: Geographical coordinates for each station, essential for spatial mapping and analysis.

#### 🚇 NYC Subway Station Dataset <br>
A dataset listing all subway and Staten Island Railway stations, with information on:
- 🌍 Location Information: Provides exact geographical locations of each station.
- 🔢 Station Master Reference Number (MRN): A unique code for each station, crucial for identification within MTA's network.
- 🔗 Complex MRN: Identifier for station complexes, highlighting interconnected stations.
- 🆔 GTFS Stop ID: Integrates stations into wider transit datasets using the Global Transit Feed Specification system.
- 🚆 Services: Lists MTA services stopping at each station, reflecting operational diversity.
- 🏗️ Station Structure Type: Categorizes stations by structure (e.g., Elevated, Underground), offering insight into architectural and logistical aspects.
- ♿ ADA-Accessibility Status: Indicates compliance with the Americans with Disabilities Act, essential for evaluating station accessibility.

#### Choice for Heavier Grading on Data Processing or Data Analysis  :
Our project is primarily concerned with data analysis, but it also heavily emphasizes the use of advanced data processing as a core component.  
Our approach goes beyond the fundamentals in data processing by using spatial analysis to optimize data merging for effectiveness.We guarantee accuracy in data alignment by utilizing sophisticated methods, like geographical coordinate-based dataset comparison and joining. Moreover, the user experience is elevated by our usage of various interactive visualizations, which improve interpretability and engagement. This offers a comprehensive and perceptive examination of the dataset, going beyond traditional data analysis.


# Phase 1: WanderQuest
Strolling through the WanderQuest phase involves importing and tidying up data. In this phase we were able to complete our project proposal task until Data Merging. In our next phase, we will continue our journey with Data Merging.

Given our datasets substantial size of millions of observations, we'll tailor the processing to suit our requirements. We'll utilize only the relevant subset of the data by focusing on a recent one year period.



In [1]:
# Importing the required libraries:
import pandas as pd
import numpy as np
import datetime
import re
import dash
from dash import dcc
from dash import html
from dash.dependencies import Input, Output
import plotly.express as px
from scipy.spatial.distance import cdist
import matplotlib.pyplot as plt


Our collection contains vital information about subway stations, including their geographic locations, structural characteristics, and accessibility aspects. Prepare to explore the city's transit environment with these various data points!

In [2]:
# Read all the data sources
stations_df = pd.read_csv('https://storage.googleapis.com/budt-2023-0503-04/MTA_Subway_Stations_20231106.csv')
riders_df = pd.read_csv('https://storage.googleapis.com/budt-2023-0503-04/MTA_Subway_Hourly_Ridership__Beginning_February_2022_20231106.csv')


# Scrape off public holidays and observance like christmas, valentine's day, etc from 1st april 2022 to 31st march 2023
holiday_2022_og = pd.read_html('https://www.timeanddate.com/holidays/us/2022', header=0)[0]
holiday_2023_og = pd.read_html('https://www.timeanddate.com/holidays/us/2023', header=0)[0]



Continuing our investigation, we now present the Subway Hourly Ridership DataFrame. It records the city's heartbeat through the comings and goings of subway commuters and is loaded with hourly data. This information, together with our station data, lays the groundwork for a full story of urban transit. Stay tuned as we delve deeper into the rhythms and patterns of metro life!

# Data Processing Steps

The following steps outline the data processing performed on two datasets: `stations_df` and `riders_df`.

## Processing `stations_df`
1. **Column Removal**: Redundant columns such as 'ADA', 'ADA Notes', 'North Direction Label', 'South Direction Label', and 'Georeference' are removed. The 'Georeference' column, being a derived column from 'Longitude' and 'Latitude', is not needed.
   
2. **Reindexing**: The DataFrame is reindexed to use 'Station ID' as the key, facilitating easier access and manipulation based on station identifiers.

## Processing `riders_df`
1. **Null Row Removal**: All rows containing null values are identified and removed. This step ensures data completeness and integrity.

2. **Column Removal**: Similar to `stations_df`, unnecessary columns like 'Georeference' and 'itsuid' are dropped from `riders_df`.

3. **Regex Application on Timestamp**: A regular expression pattern (`\d{2}\/\d{2}\/\d{4} \d{2}:\d{2}:\d{2} [AP]M`) is applied to the 'transit_timestamp' column. This process creates a boolean series indicating whether each timestamp matches the specified format.

4. **Timestamp Processing**: Since converting all timestamps to pandas datetime format for 12 million records is inefficient, the 'transit_timestamp' is split into two new columns, 'Date' and 'Time'. This step simplifies the manipulation and analysis of time-related data.


In [ ]:
# Data Processing

# Remove redeuntant column : ADA, ADA Notes. and Direction Labels.
# Remove Georeference as that is derived column made with Longitude + Latitude
stations_df_edited = stations_df.drop(columns=['ADA', 'ADA Notes', 'North Direction Label', 'South Direction Label', 'Georeference'])

# Reindex to station ID as key
stations_df_edited.set_index('Station ID')

# Check for null rows and drop those rows.
riders_df_edited = riders_df.dropna().copy()

# dropping the Georefercne, itsuid columns.
riders_df_edited.drop(columns=['Georeference', 'itsuid'], inplace=True)

# Apply the regex to the 'timestamp' column to create a boolean series
pattern = r'\d{2}\/\d{2}\/\d{4} \d{2}:\d{2}:\d{2} [AP]M'
matches_format = riders_df_edited['transit_timestamp'].str.match(pattern)

# conver all timestamp to pandas data time will be a slow process as converting 12 million records is not an optimized way.
# splitting the timestamp to time and date.
riders_df_edited[['Date', 'Time']] = riders_df_edited['transit_timestamp'].str.split(' \d', expand=True)

# Filtering and Categorizing Ridership Data

The code snippet focuses on processing the `riders_df_edited` DataFrame to filter and categorize ridership data based on specific time periods within a given date range.

## Date Filtering
- **Date Range**: The data is filtered for the period from April 1, 2022, to March 31, 2023.
- **Date Formatting**: The 'Date' column is split into 'year' and 'month_day' to facilitate filtering.
- **Filter Application**: The DataFrame is filtered to include records within the specified date range, using conditions on the 'year' and 'month_day' columns.

## Time Correction and Conversion
- **Time Correction**: The 'Time' column is corrected for midnight and noon representations ('0:00:00 AM' to '12:00:00 AM' and '0:00:00 PM' to '12:00:00 PM').
- **Time Conversion**: The corrected 'Time' values are then converted to datetime objects for easier manipulation.

## Categorization of Time Periods
- **Function Definition**: A function `categorize_time_period` is defined to categorize time into four periods:
  - 'Morning Rush' (6 AM to 12 PM)
  - 'Afternoon Rush' (12 PM to 5 PM)
  - 'Evening Rush' (5 PM to 9 PM)
  - 'Rest of the Day' (9 PM to 6 AM).
- **Application**: This function can be applied to the DataFrame to categorize each record based on its time, aiding in the analysis of ridership patterns during different times of the day.


In [ ]:
start_date = '04/01/2022'
end_date = '03/31/2023'

# Format the start and end dates to match the DataFrame's date format
start_date_formatted = pd.to_datetime(start_date, format='%m/%d/%Y')
end_date_formatted = pd.to_datetime(end_date, format='%m/%d/%Y')

riders_df_edited['year'] = riders_df_edited['Date'].str.slice(start=6, stop=10)
riders_df_edited['month_day'] = riders_df_edited['Date'].str.slice(start=0, stop=5)
riders_df_edited.drop(columns=['Date'], inplace=True)

filtered_ridership_df = riders_df_edited[((riders_df_edited['year'] == '2022') & (riders_df_edited['month_day'] >= '04/01')) |
                 ((riders_df_edited['year'] == '2023') & (riders_df_edited['month_day'] <= '03/31'))].copy()

# Format some timstamps correctly
filtered_ridership_df['Time'] = filtered_ridership_df['Time'].str.replace('0:00:00 AM', '12:00:00 AM')
filtered_ridership_df['Time'] = filtered_ridership_df['Time'].str.replace('0:00:00 PM', '12:00:00 PM')

# convert time to datetime object
filtered_ridership_df['Time'] = pd.to_datetime(filtered_ridership_df['Time'], format='%I:%M:%S %p').dt.time

# Function to categorize time into periods
def categorize_time_period(time):
    # Period 1 : 6am to 12pm -- Morning rush
    if time >= datetime.time(6, 0) and time < datetime.time(12, 0):
        return 'Morning Rush'
    # Period 2 : 12pm to 5pm -- Afternoon rush
    elif time >= datetime.time(12, 0) and time < datetime.time(17, 0):
        return 'Afternoon Rush'
    # Period 3 : 5pm to 9pm -- Evening Rush
    elif time >= datetime.time(17, 0) and time < datetime.time(21, 0):
        return 'Evening Rush'
    # Period 4 : 9pm to 6am -- Rest of the day
    else:
        return 'Rest of the Day'

# Merging for Ridership Analysis

The code performs complex data processing and merging operations on the `filtered_ridership_df` DataFrame to analyze ridership data in relation to station data. The following steps are executed:

## Categorization of Time Periods
- **Time Period Categorization**: The `categorize_time_period` function is applied to the 'Time' column, adding a new 'Time Period' column to `filtered_ridership_df`.
- **Timestamp Conversion**: The 'transit_timestamp' column is converted to a pandas datetime object for filtering.

## Data Filtering
- **Filtering Based on Date Range**: The DataFrame is filtered to include records within the specified date range using the `between` method.

## Geographical Data Processing
- **Grouping and Aggregating**: Data is grouped by 'station_complex_id' and 'station_complex', aggregating latitude and longitude to their mean values.
- **Finding Nearest Points**: A function `find_nearest_point` is defined to find the nearest station point for each row in `df_visual`, based on latitude and longitude.

## DataFrame Merging
- **Joining DataFrames**: The `find_nearest_point` function is applied to `df_merge` to join it with `stations_df_edited`, creating `joined_df2`.
- **Concatenation**: `df_merge` and `joined_df2` are concatenated to form `result_df`.
- **Merging with Station Data**: `result_df` is merged with `stations_df_edited` based on latitude and longitude, creating `station_df`.
- **Final Merge**: `df_visual` is merged with `station_df` on 'station_complex_id', resulting in `merged_df`.

## Additional Data Processing
- **Datetime Conversion**: A new 'datetime' column is created in `merged_df` by converting the 'year', 'month_day', and 'Time' columns to a datetime format.
- **Day of Week Column**: A new 'Day of Week' column is added, which contains the name of the day for each date.
- **Weekday/Weekend Categorization**: A 'Weekday/Weekend' column is created, categorizing each day as either a 'Weekend' or a 'Weekday'.
- **Aggregation and Sorting**: Finally, the DataFrame is grouped by 'station_complex_id' and 'Stop Name', aggregating 'ridership' and sorting the results in descending order.

Through these steps, the data is enriched and structured to enable in-depth analysis of ridership patterns in relation to geographical locations and time factors.


In [ ]:
# Apply function to a Time Period
filtered_ridership_df['Time Period'] = filtered_ridership_df['Time'].apply(categorize_time_period)
filtered_ridership_df['transit_timestamp'] = pd.to_datetime(filtered_ridership_df['transit_timestamp'])
df_visual = filtered_ridership_df[filtered_ridership_df['transit_timestamp'].between(start_date_formatted, end_date_formatted)]

df_merge=df_visual.groupby(['station_complex_id','station_complex']).agg({'latitude': 'mean','longitude': 'mean'})

# Function to find the nearest point in stations_df_edited for each point in df_visual
def find_nearest_point(row, df2):
    distances = cdist([(row['latitude'], row['longitude'])], df2[['GTFS Latitude', 'GTFS Longitude']])
    idx_min_distance = distances.argmin()
    nearest_point = df2.iloc[idx_min_distance]
    return pd.Series(nearest_point[['GTFS Latitude', 'GTFS Longitude']])

# Apply the function to create a new DataFrame with the joined data
joined_df2 = df_merge.apply(find_nearest_point, axis=1, df2=stations_df_edited)

# Rename columns for clarity
#joined_df2.columns = ['Nearest_Latitude', 'Nearest_Longitude']

# Concatenate the original df_visual and the joined_df
result_df = pd.concat([df_merge, joined_df2], axis=1)

result_df=result_df.reset_index()
station_df = pd.merge(result_df, stations_df_edited, left_on=['GTFS Latitude','GTFS Longitude'], right_on=['GTFS Latitude','GTFS Longitude'], how='left')

# Merge based on the new station_complex_id column
merged_df = pd.merge(df_visual, station_df, left_on='station_complex_id', right_on='station_complex_id', how='left')

# Convert 'Time' to datetime format
merged_df['datetime'] = pd.to_datetime(merged_df['year'].astype(str) + '-' + merged_df['month_day'].astype(str) + ' ' + merged_df['Time'].astype(str), format='%Y-%m/%d %H:%M:%S')
# Create a new column 'Day of Week'
merged_df['Day of Week'] = merged_df['datetime'].dt.day_name()

# Create a new column 'Weekday/Weekend'
merged_df['Weekday/Weekend'] = merged_df['datetime'].dt.day_name().apply(lambda x: 'Weekend' if x in ['Saturday', 'Sunday'] else 'Weekday')
merged_df.groupby(['station_complex_id','Stop Name']).agg({'ridership': 'sum'}).sort_values(by='ridership',ascending=False).reset_index()



# Phase 2: VisualVoyage

## Ridership Distribution by Subway Line

### Preface
This investigation looks at trends of subway ridership in each line. Comprehending the reasons behind the elevated passenger density on these lines illuminates the dynamics of urban commuting.


In [ ]:
df=merged_df
# Calculate average ridership for each subway line and time period
average_ridership_data = df.groupby(['Time Period', 'Line'])['ridership'].mean().reset_index()


time_period_order = ['Morning Rush', 'Afternoon Rush', 'Evening Rush', 'Rest of the Day']

# Create a heatmap for average ridership
fig = px.imshow(pd.pivot_table(average_ridership_data, values='ridership', index='Time Period', columns='Line'),
                x=average_ridership_data['Line'].unique(),
                y=time_period_order,
                color_continuous_scale='Viridis',
                title='Average Ridership Heatmap')



# Add label.
fig.update_xaxes(title_text='Subway Line')
fig.update_yaxes(title_text='Time Period')
fig.update_coloraxes(colorbar_title='Ridership')




fig.show()


### Emphasis on Subway line
It's important to look at eveningtime rush hour:
- **6th Avenue Culver St Line:** This line, which connects key boroughs and covers East Manhattan, helps to increase evening density.
- **8th Avenue Line and Crosstown Line:** Higher density is a result of well-planned route linkages. With its role as a cross-borough link, it sees a spike in evening commuter traffic.

### Causes of Greater Evening Density

1. **Geographic Coverage:** More evening commuters are drawn to lines that connect boroughs.
2. **Interconnectivity:** During nighttime transfers, there is an increased density of important interchanges and connectors.
The proximity of employment centers to major business regions has an impact on evening ridership.
4. **Strategic Route Planning:** Increased passenger density is a result of well-planned routes connecting important locations.

### Final Thoughts
An examination of evening ridership on particular subway lines demonstrates the complex interplay between smart route design and commuter behavior. Also using this information the Metro officials can take decision in further planning regarding linking diffrent lines and expanding network.

# Ridership Distribution by routes

<p>
   <img src="https://storage.googleapis.com/budt704-2023/newyork.jpeg" alt="Logo" width="700"/>
</p>



### Preface

The motive behind this Visualization is to find the most crowded subway route in Newyork. 

In [ ]:
# Group by the station, route, date (year + montday), + time period
grouped_df = filtered_ridership_df.groupby(['station_complex_id', 'year', 'month_day', 'Time Period', 'routes', 'payment_method', 'borough'])\
                                   .agg({'ridership': 'sum', 'transfers': 'sum'}).reset_index()


# Aggregate the data to find the average ridership for each route and time period
average_ridership = grouped_df.groupby(['routes', 'Time Period'])['ridership'].mean().reset_index()

# Pivot the data to get 'Time Period' as columns and routes as rows for plotting
pivot_df = average_ridership.pivot(index='routes', columns='Time Period', values='ridership')

# Reorder the columns of pivot_df to match the desired time period order
pivot_df = pivot_df[time_period_order]

# Find the top 10 routes with the highest average ridership
top_routes = pivot_df.mean(axis=1).nlargest(10).index

# Plotting the line graph
fig, ax = plt.subplots(figsize=(15, 10))

# Plot a line for each of the top 20 routes
for route in top_routes:
    line_data = pivot_df.loc[route, time_period_order]
    ax.plot(pivot_df.columns, line_data, marker='o', label=route, linewidth=2.5)
    # Annotate each point on this line
    for time_period in time_period_order:
        ridership = pivot_df.loc[route, time_period]
        ax.annotate(f'{ridership:.0f}',
                    (pivot_df.columns.get_loc(time_period), ridership),
                    textcoords="offset points",
                    xytext=(0,10),
                    ha='center')

# Setting the labels and title
ax.set_xlabel('Time Period')
ax.set_ylabel('Average Ridership')
ax.set_title('Average Ridership for Different Time Periods by Route')

# Set the x-ticks to the ordered time periods
ax.set_xticks(range(len(time_period_order)))
ax.set_xticklabels(time_period_order, rotation=45)

# Add a legend for the top 20 routes
ax.legend(title='Route', loc='upper left', bbox_to_anchor=(1, 1))

plt.tight_layout()
plt.show()

### This picture shows the average ridership of the highest 10 routes during different time period.


- The "Afternoon Rush" and "Evening Rush" have higher ridership than other time periods. The highest peak of ridership is indeed during the Afternoon Rush, with the most used route reaching 22,686. This observation suggests that there may be significant travel activity in the afternoon and evening in New York City, possibly due to people returning from work or school, running errands, or other social activities that typically occur in the latter part of the day.

- The route with the highest ridership(blue route), which remains high volumn across all time periods, likely serves as the major transit routes. These routes may be the central routes in the network, possibly located in a densely populated area or a business district, where a high concentration of people rely on public transportation throughout the day.

- The morning rush seems have same level with the rush in rest of the day, which could imply that morning peak is not that obvious in subway in New York City. People may not like using subway as their transoprtaion tools in the morning.  

- The routes with consistently high ridership could be a focal point for congestion management strategies. This might include measures like fare adjustments during peak times, improved routes facilities, or targeted traffic flow management. Specific services could be targeted to meet the afternoon demand, such as express routes.

## Weekday vs Weekend: Time Period Analysis

### Preface

Interactive and user-friendly interface for exploring data on subway stations. With this method, consumers may easily look into trends in station crowdedness. Users may quickly switch between weekdays and weekends with the use of simple elements like dropdown menus and animation frames, which also provide important information regarding commuter patterns.



In [ ]:
# Aggregate the ridership data.
grouped_df = merged_df.groupby(['station_complex_id','Stop Name', 'Weekday/Weekend', 'Time Period']).agg({'ridership': 'mean','latitude_x':'mean','longitude_x':'mean'}).reset_index()

# Rounding up ridership
grouped_df['ridership']=grouped_df['ridership'].round()

# To set center for visulization.
median_lat = grouped_df['latitude_x'].median()
median_lon = grouped_df['longitude_x'].median()

# Plot map using plotly.
fig = px.scatter_mapbox(grouped_df,
                        lat='latitude_x',
                        lon='longitude_x',
                        color='ridership',
                        size='ridership',
                        hover_name='Stop Name',
                        animation_frame='Time Period',
                        color_continuous_scale='Viridis',
                        size_max=40,
                        mapbox_style="carto-positron",
                        zoom=10,
                        title='Exploring Dynamic Station Crowdedness Patterns Over Time')

fig.update_layout(updatemenus=[dict(type='buttons', showactive=False, buttons=[dict(label=value, method='animate', args=[{'frame': {'duration': 500, 'redraw': True}, 'fromcurrent': True, 'transition': {'duration': 300}}], ) for value in ['Value1', 'Value2', 'Value3']])])


#App initialization using dash.
app = dash.Dash(__name__)

# Layout definition
app.layout = html.Div([
    dcc.Dropdown(
        id='weekday-weekend-dropdown',
        options=[
            {'label': value, 'value': value}
            for value in grouped_df['Weekday/Weekend'].unique()
        ],
        value=grouped_df['Weekday/Weekend'].unique()[0],
        multi=False,
        clearable=False,
        style={'width': '50%'}
    ),
    dcc.Graph(
        id='scatter-mapbox-fig',
        figure=fig,
        style={'width': '100%', 'height': '100vh'}
        # Width and height as needed
    )
])

#Update the figure based on dropdown selection
@app.callback(
    Output('scatter-mapbox-fig', 'figure'),
    [Input('weekday-weekend-dropdown', 'value')]
)
def update_figure(selected_value):

    filtered_data = grouped_df[grouped_df['Weekday/Weekend'] == selected_value]

    updated_fig = px.scatter_mapbox(filtered_data,
                                   lat='latitude_x',
                                    lon='longitude_x',
                                    color='ridership',
                                    size='ridership',
                                    hover_name='Stop Name',
                                    animation_frame='Time Period',
                                    color_continuous_scale='Viridis',
                                    size_max=40,
                                    mapbox_style="carto-positron",
                                    zoom=10,
                                    title=f'Station Crowdedness: {selected_value}')
    # Set center for visulization.
    updated_fig.update_layout(
        mapbox=dict(
            center=dict(lat=median_lat, lon=median_lon)

        )
    )

    return updated_fig



#Run
if __name__ == '__main__':
    app.run_server(debug=True, port=8070)


1. **Strategic Location:**
   In the center of Manhattan, stations such as 32nd Street Herald Square, 42nd Street Grand Central, and 14th Street Union Square are ideally situated to serve as major entry points for commuters and tourists traveling through the vast underground network of the city.
2. **Major Interchanges:**
   Important crossroads for several subway lines, these stations are always busy with commuters switching between lines. The constant crowding of the stations is partly caused by the convergence of subway lines.
3. **Commercial and Cultural Hubs:**
   The neighborhoods that surround these stations are teeming with stores, workplaces, and historical sites. The stations' constant high foot traffic is increased by the constant stream of people drawn by their proximity to busy urban activities.

## Everyone Loves Holidays

Here's a summary of the holiday data refinement:

1. **Redundancy Removal:** Extraneous rows are removed, resulting in a more streamlined dataset.

2. **Date Formatting and Renaming:** The 'Date' column is uniformly formatted, and extraneous columns are renamed to improve readability.

3. **Related Holidays Filtering:** We concentrate on Federal Holidays and Observances that fall within our target date range, resulting in a succinct dataset.


In [ ]:
# Remove the redundant rows we got from scrapping.
# Scrape off public holidays and observance like christmas, valentine's day, etc from 1st april 2022 to 31st march 2023
holiday_2022_og = pd.read_html('https://www.timeanddate.com/holidays/us/2022', header=0)[0]
holiday_2023_og = pd.read_html('https://www.timeanddate.com/holidays/us/2023', header=0)[0]
holiday_2022 = holiday_2022_og[2:len(holiday_2022_og)-1]
holiday_2023 = holiday_2022_og[2:len(holiday_2022_og)-1]

# Rename some columns
holiday_2022 = holiday_2022.rename(columns={'Unnamed: 1': 'Day'})
holiday_2023 = holiday_2023.rename(columns={'Unnamed: 1': 'Day'})

# Delete reduntant columns
holiday_2022 = holiday_2022.drop(columns='Details')
holiday_2023 = holiday_2023.drop(columns='Details')

holiday_2022['Date'] = pd.to_datetime(holiday_2022['Date'] + ' 2022')
holiday_2023['Date'] = pd.to_datetime(holiday_2023['Date'] + ' 2022')

# Federal Holiday, Observance are the two types of Holidays we are interested in. Mask with the date range we are interested in
mask_2022 = (((holiday_2022['Type'] == 'Federal Holiday') | (holiday_2022['Type'] == 'Observance')) & (holiday_2022['Date'] >= '2022-04-01'))
holiday_2022 = holiday_2022[mask_2022].groupby('Date').agg({'Name': '; '.join }).reset_index()

mask_2023 = (((holiday_2023['Type'] == 'Federal Holiday') | (holiday_2023['Type'] == 'Observance')) & (holiday_2023['Date'] < '2022-04-01'))
holiday_2023 = holiday_2023[mask_2023].groupby('Date').agg({'Name': '; '.join }).reset_index()

# Combine the filtered holidays
holiday_df = pd.concat([holiday_2022, holiday_2023], axis=0).reset_index(drop=True)


In [ ]:
# Group by the station, route, date (year + montday), + time period
hourly_data = filtered_ridership_df
holiday_data = holiday_df
# Convert 'month_day' and 'Date' to datetime objects for merging
hourly_data['month_day'] = pd.to_datetime(hourly_data['year'].astype(str) + '-' + hourly_data['month_day'], format='%Y-%m/%d')
holiday_data['Date'] = pd.to_datetime(holiday_data['Date'])

# Merge datasets on date
merged_data = pd.merge(hourly_data, holiday_data, left_on='month_day', right_on='Date', how='left')

# Initialize the Dash app
app = dash.Dash(__name__)

# Define the layout of the app
app.layout = html.Div([
    html.H1("NYC Subway Ridership During Holidays"),
    dcc.Dropdown(
        id='holiday-dropdown',
        options=[{'label': h, 'value': h} for h in holiday_data['Name'].unique()],
        value=['New Year\'s Eve', 'Thanksgiving Day'],
        multi=True
    ),
    dcc.Graph(id='ridership-map'),
    dcc.Graph(id='ridership-line-chart')
])

# Define callback for updating the map
@app.callback(
    Output('ridership-map', 'figure'),
    [Input('holiday-dropdown', 'value')]
)
def update_map(selected_holidays):
    # Filter data based on selected holidays
    filtered_data = merged_data[merged_data['Name'].isin(selected_holidays)]

    # Plot the map
    fig_map = px.scatter_mapbox(
        filtered_data,
        lat='latitude',
        lon='longitude',
        hover_name='station_complex_id',
        color='ridership',
        size='ridership',
        zoom=10,
        center={"lat": 40.7128, "lon": -74.0060},
        mapbox_style='open-street-map'
    )
    return fig_map

# Define callback for updating the line chart
@app.callback(
    Output('ridership-line-chart', 'figure'),
    [Input('holiday-dropdown', 'value')]
)
def update_line_chart(selected_holidays):
    # Filter data based on selected holidays
    filtered_data = merged_data[merged_data['Name'].isin(selected_holidays)]

    # Group by date and aggregate ridership
    grouped_data = filtered_data.groupby(['month_day', 'Name']).agg({'ridership': 'sum'}).reset_index()

    # Plot the line chart
    fig_line = px.line(
        grouped_data,
        x='month_day',
        y='ridership',
        color='Name',
        markers=True,
        title='Ridership Comparison on Selected Holidays'
    )
    return fig_line

# Run the app
if __name__ == '__main__':
    app.run_server(debug=True, port=8071)


- **Optimized Resource Management**: Understanding ridership patterns during holidays helps in efficient staffing and resource allocation at subway stations. This ensures enhanced passenger experience and operational efficiency.
- **Targeted Marketing Campaigns**: Insights from holiday ridership trends enable businesses to tailor marketing and promotional activities. Retailers and service providers in subway stations can strategize offers and advertisements to maximize foot traffic and sales.
- **Strategic Planning**: For city planners and transit authorities, this data is crucial for long-term infrastructure development, maintenance scheduling, and service optimization.

- **Interactive Data Visualization**:
  - **Map View**: Provides a geographical perspective of ridership, highlighting which stations are most frequented during different holidays. This can guide businesses in choosing optimal locations for advertisements or new ventures.
  - **Comparative Analysis**: The line chart comparison of ridership across various holidays offers a clear view of peak usage times, critical for planning and strategy development.
- **User-Driven Insights**:
  - **Customizable Selections**: The dropdown menu allows users to select specific holidays for analysis. This feature enables tailored insights, helping businesses focus on relevant data.

# Payment Analysis
### Explores key facets of public transportation payment systems

In [ ]:
grouped_df = filtered_ridership_df.groupby(['station_complex_id', 'year', 'month_day', 'Time Period', 'routes', 'payment_method', 'borough'])\
                                   .agg({'ridership': 'sum', 'transfers': 'sum'}).reset_index()
grouped_df_pivoted = grouped_df.pivot_table(index="borough", columns="payment_method", values="ridership")

# Get the borough and payment method columns
borough = grouped_df_pivoted.index
payment_method = grouped_df_pivoted.columns

# Create a stacked bar plot
plt.figure(figsize=(10, 6))
grouped_df_pivoted.plot(kind="bar", stacked=True)

# Add labels and title
plt.xlabel("Borough")
plt.ylabel("Avverage Ridership")
plt.title("Ridership by Borough and Payment Method (Stacked Bar Plot)")

# Show the plot
plt.show()

### Observation:

- **Metro Card Longevity:** - Due to their lengthy history, Metro cards are well-known among commuters.

- **Metro Card Physical Accessibility:** - Metro cards are physically available, which promotes inclusivity by allowing those without smartphones to use them.

- **Privacy and Security Issues with OMNY:** - A few commuters have expressed concerns over the digital OMNY card's privacy and security.

- **Higher OMNY Adoption in Manhattan:** OMNY has a higher percentage in Manhattan, perhaps as a result of its importance as a major business and tourism hub.


### Scope: 
- **Improved User Experience and Accessibility:**
  - Transportation authorities can enhance payment systems by considering commuter preferences and concerns. This approach ensures a smooth and inclusive experience for all users, including those without smartphones.

- **Strategic Urban Planning:**
  - Understanding the evolving dynamics of technology adoption allows urban planners to strategically allocate resources and plan infrastructure upgrades. Insights into borough-specific adoption patterns contribute to effective planning that meets the changing needs of different parts of the city.


# Conclusion


In conclusion, the analysis of New York subway data plays a pivotal role in enhancing operational efficiency and commuter experience.
- It enables the Metropolitan Transportation Authority (MTA) to effectively allocate resources, manage crowds, and make informed strategic decisions about service expansion and station upgrades.
- Furthermore, it provides a basis for targeted marketing strategies, aligning business efforts with peak commuter activity.
- Additionally, we got to know which routes, lines and stations are used most by passengers, it will definitely help many industries to plam there business tactics.
- Ultimately, this comprehensive understanding of usage patterns is instrumental in improving the overall safety, comfort, and satisfaction of commuters in New York City's subway system.